# Пример ансамбля bagging

Импорт необходимых библиотек

In [1]:
import sys

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

sys.path.append('../../')

from core.datasets import open_f
from core.ensembles import get_bagging_pred, print_bagging_ensemble_statistic

c:\Users\Mi\Desktop\ML\.conda\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Загружаем валидационный датасет, чтоб посмотреть метрики

In [2]:
val_ds = open_f('repaired_data_train', back=2)
shuffle = np.random.permutation(val_ds['labels'].shape[0])
val_ds_y = val_ds['labels'][shuffle][:7000]
val_ds_x = val_ds['images'][shuffle][:7000] / 255.

Скачиваем модели

In [3]:
model_1 = tf.keras.models.load_model(f'../checkpoints/model_g_1_categorical_accuracy.h5')
model_4 = tf.keras.models.load_model(f'../checkpoints/model_s_smote_categorical_accuracy.h5')
model_8 = tf.keras.models.load_model(f'../checkpoints/model_g_8_categorical_accuracy.h5')

models = [model_1, model_4, model_8]
names = ['model G1', 'model s_smote', 'model G8']
ensemble_model_name = 'wunderwafel_b_test1'

было замечено что повышение итогового accuracy на пару тысячных может на кегли может увеличить результат на пару сотых. Так что не стоит делать выводы только по accuracy. 

In [18]:
print_bagging_ensemble_statistic(models=models, names=names, x_val=val_ds_x, y_val=val_ds_y)

ACCURACY SCORE
model G1: 0.981
model G2: 0.9907
model G8: 0.985

ensemble bagging: 0.9947


делаем тестовый предикт и сохраняем:

Загружаем тестовый датасет, чтоб посмотреть метрики


In [ ]:
test_ds = open_f('repaired_data_test', back=2)['images'] / 255

In [5]:
wunderwafel_test = get_bagging_pred(models=models, x=test_ds)

In [6]:
ans = pd.DataFrame({'Id': np.arange(wunderwafel_test.shape[0]), 'Category': wunderwafel_test})
ans.to_csv(f"../answers/{ensemble_model_name}.csv", index=False)